<a href="https://colab.research.google.com/github/thehungtran/FFDnet_clone/blob/main/ffdnet_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Imports requirements
Allow GPU by *Execution - Modify execution type - GPU*

In [1]:
from google.colab import drive
import os, glob
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from glob import glob
from skimage.metrics import structural_similarity as ssim
import cv2


# 1. Run a test

In [2]:
! python /content/FFDNet_projet/test_ffdnet_ipol.py \
	--input /content/FFDNet_projet/test/lely.png \
	--noise_sigma 25 \
	--add_noise True

python3: can't open file '/content/FFDNet_projet/test_ffdnet_ipol.py': [Errno 2] No such file or directory


# 2. Training

## 1. Prepare dataset

In [3]:
! python /content/FFDNet_projet/prepare_patches.py \
	--trainset_dir /content/FFDNet_projet/training/ \
	--valset_dir /content/FFDNet_projet/test/ \
  --gray \
  --max_number_patches 128 # this can be suppressed, I just set it to run a quick training


python3: can't open file '/content/FFDNet_projet/prepare_patches.py': [Errno 2] No such file or directory


In [ ]:
drive.mount('Drive')

In [ ]:
! mkdir /content/Drive/My\ Drive/ffdnetmodel/ #network weights are stored here

In [ ]:
! pip install tensorboardX

In [ ]:
! python /content/FFDNet_projet/train.py \
	--batch_size 128 \
	--epochs 80 \
	--noiseIntL 0 75 \
	--val_noiseL 25 \
  --log_dir /content/Drive/My\ Drive/ffdnetmodel/ \
  --gray

**NOTES**
* The training process can be monitored with TensorBoard as logs get saved
in the *log_dir* folder
* By default, models are trained for values of noise in [0, 75] (*--noiseIntL*
flag)
* By default, noise added at validation is set to 20 (*--val_noiseL* flag)
* A previous training can be resumed passing the *--resume_training* flag